In [159]:
import pandas as pd
import historic
from pathlib import Path
import cityRemap

## import testset

In [109]:
testset = pd.read_csv(Path('extracted/high/AMS.csv'))
testset['dt'] = pd.DatetimeIndex(testset.dt)
testset = testset[testset['dt'].dt.year==2020]
testset = testset[['dt','Temperature']]

## get date range for tbats

In [134]:
df = pd.DataFrame(
        {'dt': pd.date_range('2021-01-01', '2022-01-01', freq='1H', closed='left')})

In [7]:
date = pd.DataFrame()
for month in range(1,13):
    month_data = df[df.dt.dt.month==month]
    date = date.append(month_data.iloc[0:24],ignore_index=True)

In [8]:
ams_tbats = pd.read_excel('AMSL_ExportGood.xlsx')
ams_tbats = ams_tbats[6:]
ams_tbats = ams_tbats.set_index(df.dt)
ams_tbats = ams_tbats.rename(columns={'pointForcast':'Temperature'})

## get tbats interval

In [9]:
tbats_interval = pd.DataFrame()
for month in range(12):
        for hour in range(24):
            points = ams_tbats.loc[(ams_tbats.index.hour==hour) & (ams_tbats.index.month==month+1)]
            lower = points['Temperature'].quantile(0)
            higher = points['Temperature'].quantile(1)
            tbats_interval = tbats_interval.append({'Lower':lower,'Higher':higher}, ignore_index=True)
tbats_interval = tbats_interval.set_index(date.dt)

## get histrical interval

In [110]:
historic_interval = pd.DataFrame()
historic_data = pd.read_csv('extracted/high/AMS.csv')
historic_data.dt = pd.DatetimeIndex(historic_data.dt)
historic_data = historic_data[(historic_data.dt.dt.year>2014) & (historic_data.dt.dt.year<2021)]
for month in range(1,13):
    for hour in range(24):  
            points = historic_data.loc[(historic_data.dt.dt.hour==hour) & (historic_data.dt.dt.month==month)]
            lower = points['Temperature'].quantile(0.025)
            higher = points['Temperature'].quantile(0.975)
            historic_interval = historic_interval.append({'Lower':lower,'Higher':higher}, ignore_index=True)
historic_interval = historic_interval.set_index(date.dt)

# evaluate performance

In [54]:
def result(interval_set,offset):
    correct = 0
    total = 0
    old_correct = 0
    for hour in range(24):
        for month in range(1,13):
            upper,lower = interval_set[(interval_set.index.hour==hour) & (interval_set.index.month==month)].iloc[0].values
            upper += offset
            lower -= offset
            points = testset.loc[(testset.dt.dt.hour==hour) & (testset.dt.dt.month==month)]
            total += len(points)
            correct += points['Temperature'].between(lower,upper).sum()
        old_correct = correct
    return correct/total

In [56]:
print(result(tbats_interval,8))

0.8616803278688525


In [61]:
print(result(historic_interval,0))

0.951844262295082
